# Imports and Data

In [3]:
import pandas as pd
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json

In [2]:
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("detach", True)
ser = Service("C:\Program Files (x86)\chromedriver.exe")

In [3]:
source_data = pd.read_excel("../../Data/ERA_Alabama.xlsx",sheet_name="All Plants")

# Scrape 1 plant

In [10]:
driver = webdriver.Chrome(service=ser, options=options)
driver.get("https://www.gardenia.net/plant/ficus-benjamina")
source = driver.page_source

In [13]:
tables = pd.read_html(source)

In [20]:
tables[0]

,0,1
0,Hardiness,10 – 12 What's My Zone?
1,Heat Zones,10 – 12 What's My Zone?
2,Climate Zones,"13, 23, 24, H1, H2"
3,Plant Type,Trees
4,Plant Family,Ficus
5,Exposure,"Full Sun, Partial Sun"
6,Season of Interest,"Spring (Early,Mid,Late)Summer (Early,Mid,Late)..."
7,Height,40' – 50' (12m – 15m)
8,Spread,25' – 30' (7.5m – 9m)
9,Water Needs,Average


In [28]:
table = tables[0].set_index(0).T

In [30]:
table.index = ["name of plant"]

In [31]:
table

,Hardiness,Heat Zones,Climate Zones,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Native Plants,Tolerance,Garden Uses,Garden Styles
name of plant,10 – 12 What's My Zone?,10 – 12 What's My Zone?,"13, 23, 24, H1, H2",Trees,Ficus,"Full Sun, Partial Sun","Spring (Early,Mid,Late)Summer (Early,Mid,Late)...",40' – 50' (12m – 15m),25' – 30' (7.5m – 9m),Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral",Moist but Well-Drained,"Plant of Merit, Showy, Evergreen, Fruit & Berries",Australia,Drought,Patio and Containers,Mediterranean Garden


# Scrape sample of plants and join 

Assumes tables are of the same structure

In [4]:
names = source_data["Scientific Name"]
names = ["-".join(name.lower().split(" ")) for name in names]
names[:5]

['acacia-farnesiana',
 'acalypha-rhomboidea',
 'acalypha-virginica',
 'acer-floridanum',
 'acer-negundo']

In [5]:
names_short = names

for name in names_short:
    response = requests.get(f"https://www.gardenia.net/plant/{name}")
    print(response.status_code)
    if response.status_code != 200:
        names_short.remove(name)

200
404
200
200
404
404
404
404
404
200
200
404
200
200


In [41]:
names_short

['acacia-farnesiana',
 'acalypha-virginica',
 'acer-floridanum',
 'acer-negundo',
 'acer-rubrum',
 'acer-saccharum',
 'achillea-millefolium',
 'aconitum-uncinatum',
 'actaea-racemosa',
 'adiantum-capillus-veneris',
 'adiantum-pedatum',
 'aeschynomene-viscidula',
 'aesculus-flava',
 'ficus-benjamina']

In [20]:
driver = webdriver.Chrome(service=ser, options=options)

full_df = None
dfs = []

list_of_unique_columns = []

for name in names_short[:3]:
    driver.get(f"https://www.gardenia.net/plant/{name}")
    source = driver.page_source

    #Some pages don't exist but still return 200
    try:
        tables = pd.read_html(source)
    except ValueError:
        continue
    plant_df = tables[0].set_index(0).T
    plant_df.index = [name.replace("-"," ")]


    dfs.append(plant_df)

    for column in plant_df.columns:
        if column not in list_of_unique_columns:
            list_of_unique_columns.append(column)


driver.close()

    

In [21]:
dfs[0] #Missing attracts

,Hardiness,Climate Zones,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Native Plants,Tolerance,Garden Uses,Garden Styles
acacia farnesiana,9 – 11 What's My Zone?,"13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24","Shrubs, Trees",Acacia,"Full Sun, Partial Sun","Spring (Early,Mid)Winter",15' – 20' (4.5m – 6m),15' – 20' (4.5m – 6m),"Low, Average",Low,"Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Fragrant, Showy","United States, California, Southeast, Alabama,...","Deer, Drought, Dry Soil, Salt",Hedges and Screens,Mediterranean Garden


In [22]:
dfs[1] #Missing climate zones,garden uses

,Hardiness,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Native Plants,Tolerance,Attracts,Garden Styles
acer floridanum,6 – 9 What's My Zone?,Trees,Acer - Maples,"Full Sun, Partial Sun","Spring (Early,Mid,Late)Summer (Early,Mid,Late)...",20' – 60' (6m – 18m),25' – 40' (7.5m – 12m),Average,Low,"Clay, Loam, Sand","Acid, Neutral","Moist but Well-Drained, Well-Drained",Showy,"United States, Midwest, Illinois, Missouri, So...",Drought,Birds,"Informal and Cottage, Prairie and Meadow"


In [24]:
import numpy as np
for df in dfs:
    print(list(set(list_of_unique_columns).difference(df.columns)))

['Attracts']
['Garden Uses', 'Climate Zones']


In [29]:
import numpy as np
for df in dfs:
    for name in list(set(list_of_unique_columns).difference(df.columns)):
        df[name] = np.NaN
        df.reindex(list_of_unique_columns,axis=1)
    


In [30]:
full_df = pd.concat(dfs)

In [31]:
full_df

,Hardiness,Climate Zones,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Native Plants,Tolerance,Garden Uses,Garden Styles,Attracts
acacia farnesiana,9 – 11 What's My Zone?,"13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24","Shrubs, Trees",Acacia,"Full Sun, Partial Sun","Spring (Early,Mid)Winter",15' – 20' (4.5m – 6m),15' – 20' (4.5m – 6m),"Low, Average",Low,"Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Fragrant, Showy","United States, California, Southeast, Alabama,...","Deer, Drought, Dry Soil, Salt",Hedges and Screens,Mediterranean Garden,NaN
acer floridanum,6 – 9 What's My Zone?,NaN,Trees,Acer - Maples,"Full Sun, Partial Sun","Spring (Early,Mid,Late)Summer (Early,Mid,Late)...",20' – 60' (6m – 18m),25' – 40' (7.5m – 12m),Average,Low,"Clay, Loam, Sand","Acid, Neutral","Moist but Well-Drained, Well-Drained",Showy,"United States, Midwest, Illinois, Missouri, So...",Drought,NaN,"Informal and Cottage, Prairie and Meadow",Birds


# All names 

In [32]:
for name in names:
    response = requests.get(f"https://www.gardenia.net/plant/{name}")
    if response.status_code != 200:
        names.remove(name)

with open("Names.txt","w") as f:
    for name in names:
        f.write(name + "\n")


# Write to json (in case of error)

In [34]:
driver = webdriver.Chrome(service=ser, options=options)

full_df = None
dfs = []

list_of_unique_columns = []

for name in names_short[:3]:
    driver.get(f"https://www.gardenia.net/plant/{name}")
    source = driver.page_source

    #Some pages don't exist but still return 200
    try:
        tables = pd.read_html(source)
    except ValueError:
        continue
    plant_df = tables[0].set_index(0).T
    plant_df.index = [name.replace("-"," ")]


    dfs.append(plant_df)

    for column in plant_df.columns:
        if column not in list_of_unique_columns:
            list_of_unique_columns.append(column)

    with open("Backup.txt",'w') as f:
        json_string = json.dumps([df.to_dict() for df in dfs])
        f.write(json_string)


driver.close()


In [38]:
with open("Backup.txt") as f:
    json_string = f.read()
    dfs  = json.loads(json_string)

In [43]:
pd.DataFrame(dfs[0])

,Hardiness,Climate Zones,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Native Plants,Tolerance,Garden Uses,Garden Styles
acacia farnesiana,9 – 11 What's My Zone?,"13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24","Shrubs, Trees",Acacia,"Full Sun, Partial Sun","Spring (Early,Mid)Winter",15' – 20' (4.5m – 6m),15' – 20' (4.5m – 6m),"Low, Average",Low,"Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Fragrant, Showy","United States, California, Southeast, Alabama,...","Deer, Drought, Dry Soil, Salt",Hedges and Screens,Mediterranean Garden
